# Сбор связей

In [ ]:
!pip install pymorphy2-dicts-ru

In [ ]:
!pip install levenshtein

In [ ]:
from collections import Counter, defaultdict
from nlp import nlp
from dawg import CompletionDAWG
import torch
import stanza
import glob


def filter_deps(word):
    return word.deprel in ["amod"] # , "appos"


def find_deps(words):
    deps = []
    
    for w in words:
        if w.upos == "NOUN" and "Animacy=Anim" in w.feats:
            deps.append((w, [rw for rw in words if rw.head == w.id and filter_deps(rw)]))
    
    return deps


def format_dep(word):
    return f"{word.lemma.replace('-', '')}:{word.deprel}:{word.upos}"


def collect_deps(all_deps, deps):
    for (root, dep_words) in deps:
        if dep_words:
            all_deps[root.lemma.replace('-', '')] += Counter({ format_dep(dw) for dw in dep_words })

    
def get_stanza_filenames():
    return glob.iglob('/mnt/data/proza_ru/**/*.txt.stanza', recursive=True)


def main():
    deps_counter = defaultdict(Counter)
    
    i = 0
    
    with torch.no_grad():
        for filename in get_stanza_filenames():
            if i % 1000 == 0: print(i)
            i += 1
            
            with open(filename, 'rb') as file:
                try:
                    doc = stanza.Document.from_serialized(file.read())

                    for sent in doc.sentences:
                        collect_deps(deps_counter, find_deps(sent.words))
                except Exception as e:
                    print(f"rm {filename} &&")

    return deps_counter


# deps = defaultdict(Counter)

# for sent in doc.sentences:
#     add_deps(deps, find_deps(sent.words))
    


# completion_dawg = CompletionDAWG([f"{root}:{dep}" for root in deps for dep in deps[root]])
rels = main()

In [281]:
from morph import morph
from rapidfuzz.distance import JaroWinkler
from itertools import takewhile
import dawg


def is_known(word):
    for parse in morph.parse(word):
        if parse.normal_form.replace('ё', 'е') == word.replace('ё', 'е') and 'nomn' in parse.tag and parse.is_known and ('anim' in parse.tag or 'ADJF' in parse.tag or "PRTF" in parse.tag):
            return True

    return False


def normalize(word):
    for parse in morph.parse(word):
        if parse.is_known and ('anim' in parse.tag or 'ADJF' in parse.tag or "PRTF" in parse.tag):
            return parse.normal_form.replace('ё', 'е')

    return word.replace('ё', 'е')


# Удалить руты с очен маленьким количеством связей
# Удалить редкие депсы
def merge_unknown_words_to_known(rels):
    dict_roots = [root for root in rels if is_known(root)]
    non_dict_roots = [root for root in rels if root not in dict_roots]

    print(len(dict_roots), len(non_dict_roots))
    
    # Удаляем редкие неизвестные слова
    # и совсем короткие
    # и нормализуемые
    for ndr in non_dict_roots:
        vals = rels[ndr].values()
        
        if max(vals) < 3:
            print('Удаляем по весу ', ndr, vals)
            rels.pop(ndr)
            non_dict_roots.remove(ndr)
            
        if len(ndr) < 3:
            print('Удаляем по длине ', ndr, vals)
            rels.pop(ndr)
            non_dict_roots.remove(ndr)
        
        normalized_ndr = normalize(ndr)
        if normalized_ndr != ndr and normalized_ndr in dict_roots:
            print('Сливаем normalized', ndr, normalized_ndr)
            rels[normalized_ndr] |= rels[ndr]
            rels.pop(ndr) 
            non_dict_roots.remove(ndr)
            
        ndr_a = f'{ndr}а';
        
        if is_known(ndr_a):
            print('Сливаем +a', ndr, ndr_a)
            rels[ndr_a] = rels.get(ndr_a, Counter())
            rels[ndr_a] |= rels[ndr]
            rels.pop(ndr)
            non_dict_roots.remove(ndr)
        
            
    # Сливаем похожие слова
    for ndr in non_dict_roots:
        nearest_dr = None
        nearest_dr_sim = 0
        
        for dr in dict_roots:
            sim = JaroWinkler.similarity(ndr, dr)
            
            if sim < 0.96:
                continue
                
            if sim > nearest_dr_sim:
                nearest_dr_sim = sim
                nearest_dr = dr
        
        if nearest_dr:
            print('Сливаем близкие ', ndr, nearest_dr)
            rels[nearest_dr] |= rels[ndr]
            rels.pop(ndr)
            
    print(len(dict_roots), len(non_dict_roots))
        

def drop_rare_roots(rels):
    rare = [root for root in rels if max(rels[root].values()) < 3]
    
    for r in rare:
        print('drop_rare_roots ', r, max(rels[r].values()))
        rels.pop(r)
        
        


# merge_unknown_words_to_known(rels)


def rels_to_seq(rels):
    for root in rels:
        if not is_known(root):
            continue
            
        deps = rels[root]
            
        bound = 0
        
        if len(deps) > 2:
            [common1, common2] = deps.most_common(2)
            bound = common2[1] / 100
    
        for dep in deps:
#            местоимение
            [dep_word, dep_rel, dep_pos] = dep.split(":")
            if 'Apro' in morph.parse(dep_word)[0].tag:
                continue
                
            if 'блуд' in dep_word or 'метр' in dep_word or 'твой' in dep_word:
                continue
            
            if deps[dep] >= bound:
                yield f"{root.lower().replace('ё', 'е')}:{dep.lower().replace('ё', 'е')}"
                



completion_dawg = dawg.CompletionDAWG(rels_to_seq(rels))

In [282]:
completion_dawg.keys('пес:')

['пес:адский:amod:adj',
 'пес:андалузский:amod:adj',
 'пес:барский:amod:adj',
 'пес:бдительный:amod:adj',
 'пес:бедный:amod:adj',
 'пес:бездомный:amod:adj',
 'пес:безобидный:amod:adj',
 'пес:безродный:amod:adj',
 'пес:белый:amod:adj',
 'пес:беспородный:amod:adj',
 'пес:беспризорный:amod:adj',
 'пес:бестолковый:amod:adj',
 'пес:бешенный:amod:adj',
 'пес:бешеный:amod:adj',
 'пес:бешить:amod:verb',
 'пес:благородный:amod:adj',
 'пес:бледный:amod:adj',
 'пес:ближайший:amod:adj',
 'пес:блохой:amod:adj',
 'пес:боевой:amod:adj',
 'пес:бойцовой:amod:adj',
 'пес:бойцовский:amod:adj',
 'пес:бойцовый:amod:adj',
 'пес:больной:amod:adj',
 'пес:большой:amod:adj',
 'пес:большой:amod:propn',
 'пес:браться:amod:verb',
 'пес:бродячий:amod:adj',
 'пес:бросить:amod:verb',
 'пес:бывший:amod:adj',
 'пес:верный:amod:adj',
 'пес:веселый:amod:adj',
 'пес:взбеситься:amod:verb',
 'пес:взрослый:amod:adj',
 'пес:волшебный:amod:adj',
 'пес:вольный:amod:adj',
 'пес:воспитанный:amod:verb',
 'пес:второй:amod:adj',
 'п

True

# Распознание сущностей